# Mastering PostgreSQL with Python (Psycopg3)

This tutorial covers the transition from basic SQL execution to high-performance data engineering patterns.

### Core Concepts Covered:
1. **Modern Connectivity**: Using context managers.
2. **Schema Design**: DDL and Relationships.
3. **Performance**: Atomic `COPY` vs slow `INSERT`.
4. **NoSQL Power**: Querying `JSONB` fields.
5. **Transactions**: Ensuring ACID compliance.

In [ ]:
# 1. Environment Setup
%pip install "psycopg[binary]" pandas

In [ ]:
import psycopg
from psycopg import sql
import json
import pandas as pd

# Configuration Dictionary
DB_CONFIG = {
    "host": "localhost",
    "dbname": "postgres",
    "user": "postgres",
    "password": "your_password",
    "port": 5432
}

def get_conn():
    return psycopg.connect(**DB_CONFIG)

print("Setup complete. Ready to connect.")

## 2. Schema Design
We will create a relational structure: **Users** and **Orders**.

In [ ]:
with get_conn() as conn:
    with conn.cursor() as cur:
        # Create Users Table
        cur.execute("""
            CREATE TABLE IF NOT EXISTS users (
                user_id SERIAL PRIMARY KEY,
                username TEXT UNIQUE NOT NULL,
                email TEXT NOT NULL
            );
        """)
        
        # Create Orders Table with JSONB
        cur.execute("""
            CREATE TABLE IF NOT EXISTS orders (
                order_id SERIAL PRIMARY KEY,
                user_id INTEGER REFERENCES users(user_id),
                item TEXT,
                details JSONB
            );
        """)
        conn.commit()
print("Tables created successfully.")

## 3. High-Performance Bulk Loading
The `COPY` protocol is the professional standard for speed in PostgreSQL.



In [ ]:
users_to_add = [
    ("dev_user", "dev@example.com"),
    ("data_sci", "data@example.com"),
    ("dba_admin", "admin@example.com")
]

with get_conn() as conn:
    with conn.cursor() as cur:
        with cur.copy("COPY users (username, email) FROM STDIN") as copy:
            for user in users_to_add:
                copy.write_row(user)
    conn.commit()
print("Bulk insert completed using COPY.")

## 4. Querying JSONB Data
Query inside JSON using the `->>` (extract text) operator.

In [ ]:
order_data = (1, "Laptop", json.dumps({"specs": {"ram": "32GB", "cpu": "M3"}}))

with get_conn() as conn:
    with conn.cursor() as cur:
        cur.execute("INSERT INTO orders (user_id, item, details) VALUES (%s, %s, %s)", order_data)
        
        cur.execute("""
            SELECT item, details->'specs'->>'cpu' as processor
            FROM orders
            WHERE details->'specs'->>'ram' = '32GB';
        """)
        print("Query Result:", cur.fetchone())
    conn.commit()

## 5. Transactions & Safety
A transaction ensures that either all operations succeed, or none do.

In [ ]:
try:
    with get_conn() as conn:
        with conn.cursor() as cur:
            cur.execute("UPDATE users SET email = 'updated@example.com' WHERE user_id = 1")
            # Uncomment the next line to test rollback
            # raise ValueError("Simulated failure!")
            conn.commit()
            print("Transaction committed successfully.")
except Exception as e:
    print(f"Transaction failed! Error: {e}")